In [ ]:
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.datasets import mnist
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = mnist.load_data()

# Reshape the images to (28, 28, 1) and normalize the pixel values
train_ds = train_ds.reshape(-1, 28, 28, 1).astype("float32") / 255.0
test_ds = test_ds.reshape(-1, 28, 28, 1).astype("float32") / 255.0

# Repeat grayscale images to have 3 channels
train_ds = np.repeat(train_ds, 3, axis=-1)
test_ds = np.repeat(test_ds, 3, axis=-1)

## Resizing images
train_ds = tf.image.resize(train_ds, (32, 32))
test_ds = tf.image.resize(test_ds, (32, 32))

## Loading EfficientNetB7 model
base_model = EfficientNetB7(include_top=False, weights="imagenet", input_shape=(32, 32, 3))

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(10, activation='softmax')

model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])
model.summary()

# One-hot encode the labels
train_labels = to_categorical(train_labels, num_classes=10)
test_labels = to_categorical(test_labels, num_classes=10)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)

model.fit(train_ds, train_labels, epochs=3, validation_data=(test_ds, test_labels), batch_size=32, callbacks=[es])

scores = model.evaluate(test_ds, test_labels, verbose=0)

print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

model.trainable = True
base_model.trainable = True
model.fit(train_ds, train_labels, epochs=10, validation_data=(test_ds, test_labels), batch_size=32, callbacks=[es])

scores = model.evaluate(test_ds, test_labels, verbose=0)

print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tensorflow13GcsFileSystemE']
  warnings.warn(f"file system plugins are not loaded: {e}")

Downloading data from https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz
11490434/11490434 [==============================] - 0s 0us/step
Downloading data from https://storage.googleapis.com/keras-applications/efficientnetb7_notop.h5
258076736/258076736 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 efficientnetb7 (Functional)  (None, 1, 1, 2560)       64097687  
                                                                 
 flatten (Flatten)           (None, 2560)              0         
                                                                 
 dense (Dense)               (None, 50)                128050    
                                                                 
 dense_1 (Dense)             (None, 20)                1020      
                                                                 
 dense_2 (Dense)             (None, 10)                210       
                                                                 
=================================================================
Total params: 64,226,967
Trainable params: 63,916,240
Non-trainable params: 310,727
_________________________________________________________________
Epoch 1/3

2023-06-15 10:00:57.286814: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/efficientnetb7/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer

1875/1875 [==============================] - 385s 122ms/step - loss: 0.2467 - accuracy: 0.9380 - val_loss: 0.3603 - val_accuracy: 0.9143
Epoch 2/3
1875/1875 [==============================] - 223s 119ms/step - loss: 0.1049 - accuracy: 0.9761 - val_loss: 0.0564 - val_accuracy: 0.9871
Epoch 3/3
1875/1875 [==============================] - 225s 120ms/step - loss: 0.1057 - accuracy: 0.9749 - val_loss: 1.3726 - val_accuracy: 0.5095

accuracy: 50.95%
Epoch 1/10
1875/1875 [==============================] - 223s 119ms/step - loss: 0.0666 - accuracy: 0.9844 - val_loss: 0.0803 - val_accuracy: 0.9773
Epoch 2/10
1875/1875 [==============================] - 231s 123ms/step - loss: 0.0578 - accuracy: 0.9869 - val_loss: 0.0674 - val_accuracy: 0.9852
Epoch 3/10
1875/1875 [==============================] - 223s 119ms/step - loss: 0.0472 - accuracy: 0.9886 - val_loss: 5.7751 - val_accuracy: 0.2587
Epoch 4/10
1875/1875 [==============================] - 223s 119ms/step - loss: 0.0470 - accuracy: 0.9894 - val_loss: 0.1385 - val_accuracy: 0.9662
Epoch 5/10
1875/1875 [==============================] - 230s 123ms/step - loss: 0.0419 - accuracy: 0.9903 - val_loss: 0.0854 - val_accuracy: 0.9792
Epoch 6/10
1875/1875 [==============================] - 230s 123ms/step - loss: 0.0357 - accuracy: 0.9915 - val_loss: 17.5839 - val_accuracy: 0.0892
Epoch 7/10
1875/1875 [==============================] - 224s 120ms/step - loss: 0.0323 - accuracy: 0.9924 - val_loss: 0.0672 - val_accuracy: 0.9823

accuracy: 98.52%
